In [64]:
%pip install langgraph langchain langchain-google-genai langchain-community faiss-cpu

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [65]:
%pip install openai

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ------------------------------- -------- 0.8/1.0 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 4.8 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
from typing import TypedDict, Annotated, Sequence
from langgraph.graph import Graph, StateGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.messages import HumanMessage
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import json
import csv
from dotenv import load_dotenv
from langgraph.graph import StateGraph, END

from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from typing import List, Any, Dict, Optional
from openai import OpenAI


# Load environment variables from .env file
load_dotenv()  # take environment variables from .env file

True

In [ ]:

class NvidiaChatModel(BaseChatModel):
    client: Any = None
    model_name: str = "qwen/qwen3-next-80b-a3b-instruct"
    temperature: float = 0.6
    top_p: float = 0.7
    max_tokens: int = 4096
    api_key: str = "" # Will be loaded from env if not provided

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # Load API key from environment variable if not explicitly provided
        if not self.api_key:
            self.api_key = os.getenv("NVIDIA_API_KEY", "")
            if not self.api_key:
                raise ValueError("NVIDIA_API_KEY not found. Please set it as an environment variable or pass it to NvidiaChatModel.")

        self.client = OpenAI(
            base_url="https://integrate.api.nvidia.com/v1",
            api_key=self.api_key
        )

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[Any] = None,
        **kwargs: Any,
    ) -> ChatResult:
        # Convert LangChain messages to NVIDIA API format
        formatted_messages = []
        for msg in messages:
            if isinstance(msg, HumanMessage):
                formatted_messages.append({"role": "user", "content": msg.content})
            elif isinstance(msg, AIMessage):
                formatted_messages.append({"role": "assistant", "content": msg.content})
            elif isinstance(msg, SystemMessage):
                formatted_messages.append({"role": "system", "content": msg.content})
            # Add other message types if needed

        # Make the API call
        completion = self.client.chat.completions.create(
            model=self.model_name,
            messages=formatted_messages,
            temperature=self.temperature,
            top_p=self.top_p,
            max_tokens=self.max_tokens,
            stop=stop, # Pass stop sequences if provided
            stream=False # For _generate, typically non-streaming is used
        )

        # Extract content from the response
        response_content = completion.choices[0].message.content
        
        # Create an AIMessage
        ai_message = AIMessage(content=response_content)
        
        # Create a ChatGeneration object, which includes the AIMessage
        chat_generation = ChatGeneration(message=ai_message, text=response_content)
        
        # Wrap it in a ChatResult
        return ChatResult(generations=[chat_generation])

    @property
    def _llm_type(self) -> str:
        return "nvidia_chat_model"

In [ ]:
# Initialize NVIDIA Chat Model
llm = NvidiaChatModel(
    model_name="qwen/qwen3-next-80b-a3b-instruct",
    temperature=0.6,
    top_p=0.7,
    max_tokens=4096,
    # api_key=os.getenv("NVIDIA_API_KEY") # The class will automatically try to load it
)

# Initialize Tavily Search
tavily_tool = TavilySearchResults(api_key=os.getenv("TAVILY_API_KEY"))

In [53]:
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.Collecting pypdf




[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [72]:
# Create a vector store
def create_vector_store():
    """
    Create a vector store from PDF documents.
    
    This function loads PDF documents, splits the text into chunks, and creates embeddings using Google Generative AI.
    
    Returns:
        FAISS: A vector store containing the text and corresponding embeddings.
    """
    
    try:
        # Try to load the existing vector store first
        embeddings = GoogleGenerativeAIEmbeddings(
            model="models/gemini-embedding-exp-03-07", 
            google_api_key=os.getenv("GEMINI_API_KEY")
        )
        # vector_store = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
        # print("Loaded existing vector store.")
        # return vector_store
    except Exception as e:
        print(f"Could not load existing vector store: {e}. Creating new vector store...")
    
    # Loading documents
    documents = []
    try:
        for i in range(1, 4):
            try:
                loader = PyPDFLoader(f"document{i}.pdf")
                documents.extend(loader.load())
            except FileNotFoundError:
                print(f"File document{i}.pdf not found, skipping.")
    except Exception as e:
        print(f"Error loading documents: {e}")
        # Fallback to loading just the first document
        try:
            loader = PyPDFLoader("document_1.pdf")
            documents.extend(loader.load())
        except FileNotFoundError:
            print("Critical: No documents found")
            return None
    
    # Splitting documents and creating text representations
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    texts = text_splitter.split_documents(documents)
    print(texts)
    print(f"Number of text chunks: {len(texts)}")
    
    # Create embeddings using a Google Generative AI model
    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/gemini-embedding-exp-03-07", 
        google_api_key=os.getenv("GEMINI_API_KEY")
    )
    
    # Create a vector store using FAISS from the provided text chunks and embeddings
    vector_store = FAISS.from_documents(texts, embedding=embeddings)
    
    # Save the vector store locally with the name "faiss_index"
    vector_store.save_local("faiss_index")
    return vector_store


# Initialize vector store
try:
    vector_store = create_vector_store()
    if vector_store is None:
        raise ValueError("Failed to create vector store")
except Exception as e:
    print(f"Error creating vector store: {e}")
    # # Create a simple dummy vector store for testing
    # vector_store = FAISS.from_texts(["Dummy text about malaria for testing"], 
    #                                embedding=GoogleGenerativeAIEmbeddings(
    #                                    model="models/gemini-embedding-exp-03-07",
    #                                    google_api_key=os.getenv("GEMINI_API_KEY")
    #                                ))


[Document(metadata={'source': 'document1.pdf', 'page': 0, 'page_label': '1'}, page_content="Disease Information - Document1\nDisease Name: Malaria\nDescription: Malaria is a serious and sometimes fatal disease caused by a parasite that commonly\ninfects a certain type of mosquito which feeds on humans. People who get malaria are typically very\nsick with high fevers, shaking chills, and flu-like illness.\nICD Code: B54\nDisease Name: Tuberculosis\nDescription: Tuberculosis (TB) is a potentially serious infectious disease that mainly affects your\nlungs. The bacteria that cause tuberculosis are spread from person to person through tiny droplets\nreleased into the air via coughs and sneezes.\nICD Code: A15.0\nDisease Name: Pneumonia\nDescription: Pneumonia is an infection that inflames air sacs in one or both lungs. The air sacs may fill\nwith fluid or pus (purulent material), causing cough with phlegm or pus, fever, chills, and difficulty\nbreathing.\nICD Code: J18.9\nDisease Name: Diab

In [84]:
# Create a search function for the vector store
def search_vector_store(query: str) -> str:
    try:
        print(f"Starting vector search for query: {query}")
        results = vector_store.similarity_search(query, k=1)
        print(f"Vector search results: {results}")
        return results[0].page_content
    except Exception as e:
        print(f"Error searching vector store: {e}")
        return "No information found in the vector store."
    
    
# Create a Tavily search function   
def execute_tavily_tool(query: str):
    try:
        print(f"Starting Tavily search for query: {query}")
        results = tavily_tool.run(query)
        print(f"Tavily search results: {results}")
        return results
    except Exception as e:
        print(f"Error in Tavily search: {e}")
        return "No information found using Tavily."
    
# Create a tool for the vector store
vector_search_tool = Tool(
    name="VectorSearch",
    func=search_vector_store,
    description="Searches the vector store for relevant information about diseases."
)

# Create a tool for Tavily search
tavily_web_search_tool =  Tool(
        name="TavilySearch",
        func=execute_tavily_tool,
        description="Searches the web for relevant information about diseases."
    )

# Store tools in a list
tools = [vector_search_tool, tavily_web_search_tool]


In [74]:
search_vector_store("Gallstones")

Starting vector search for query: Gallstones
Vector search results: [Document(id='717f54e7-3be6-4619-a423-833284645f2a', metadata={'source': 'document3.pdf', 'page': 0, 'page_label': '1'}, page_content="Disease Information - Document3\nDisease Name: Gout\nDescription: Gout is a common and complex form of arthritis that can affect anyone. It's characterized\nby sudden, severe attacks of pain, swelling, redness and tenderness in one or more joints, most often\nin the big toe.\nICD Code: M10.9\nDisease Name: Kidney Stones\nDescription: Kidney stones (also called renal calculi, nephrolithiasis or urolithiasis) are hard deposits\nmade of minerals and salt that form inside your kidneys. Kidney stones can affect any part of your\nurinary tract.\nICD Code: N20.0\nDisease Name: Gallstones\nDescription: Gallstones are hardened deposits of digestive fluid that can form in your gallbladder. Your\ngallbladder is a small, pear-shaped organ on the right side of your abdomen, just beneath your liver.\

"Disease Information - Document3\nDisease Name: Gout\nDescription: Gout is a common and complex form of arthritis that can affect anyone. It's characterized\nby sudden, severe attacks of pain, swelling, redness and tenderness in one or more joints, most often\nin the big toe.\nICD Code: M10.9\nDisease Name: Kidney Stones\nDescription: Kidney stones (also called renal calculi, nephrolithiasis or urolithiasis) are hard deposits\nmade of minerals and salt that form inside your kidneys. Kidney stones can affect any part of your\nurinary tract.\nICD Code: N20.0\nDisease Name: Gallstones\nDescription: Gallstones are hardened deposits of digestive fluid that can form in your gallbladder. Your\ngallbladder is a small, pear-shaped organ on the right side of your abdomen, just beneath your liver.\nICD Code: K80.20\nDisease Name: Osteoporosis\nDescription: Osteoporosis causes bones to become weak and brittle — so brittle that a fall or even\nmild stresses such as bending over or coughing can caus

In [75]:
execute_tavily_tool("ICD code Gallstones")

Starting Tavily search for query: ICD code Gallstones
Tavily search results: [{'url': 'https://gesund.bund.de/en/icd-code-search/k80-20', 'content': 'The portal uses cookies to provide service functions such as “Bookmark” and to improve website usage. Information and opt-out options can be found in the cookie settings in the privacy policy.\n\nSkip navigation\n\n Deutsch\n English\n Türkçe\n Русский\n عربي\n\nSprache Language Dilim Язык اللغة\n\n Search\n\n K80.2\n K80.20\n\n# ICD-Code K80.20: Calculus of gallbladder without cholecystitis Without mention of obstruction of biliary tract\n\n## You have gallstones in your gallbladder. [...] This information is not intended for self-diagnosis and does not replace professional medical advice from a doctor. If you find an ICD code on a personal medical document, please also note the additional indicator used for diagnostic confidence.Your doctor will assist you with any health-related questions and explain the ICD diagnosis code to you in a 

[{'url': 'https://gesund.bund.de/en/icd-code-search/k80-20',
  'content': 'The portal uses cookies to provide service functions such as “Bookmark” and to improve website usage. Information and opt-out options can be found in the cookie settings in the privacy policy.\n\nSkip navigation\n\n Deutsch\n English\n Türkçe\n Русский\n عربي\n\nSprache Language Dilim Язык اللغة\n\n Search\n\n K80.2\n K80.20\n\n# ICD-Code K80.20: Calculus of gallbladder without cholecystitis Without mention of obstruction of biliary tract\n\n## You have gallstones in your gallbladder. [...] This information is not intended for self-diagnosis and does not replace professional medical advice from a doctor. If you find an ICD code on a personal medical document, please also note the additional indicator used for diagnostic confidence.Your doctor will assist you with any health-related questions and explain the ICD diagnosis code to you in a direct consultation if necessary.\n\n## Source'},
 {'url': 'https://www.icd

In [85]:
# Define the agent's prompt
prompt = PromptTemplate.from_template(
    """You are a medical information retrieval agent. Your task is to find information about diseases and their ICD codes.

    Tools you can use:
    {tools}

    Tool name: {tool_names}

    Human: {human_input}
    You must respond in the following format:
        Thought: <Your thought process>
        Action: <The action you will take>
        Action Input: <The input for the action>
        Observation: <The result of the action>
        ... (this Thought/Action/Action Input/Observation can repeat N times)
        Thought: I have gathered all the necessary information and will now provide the final answer.
        Final Answer: <The final answer in JSON format>

    Always use the following steps:
    1. **Retrieve Disease Description:** Use VectorSearch to retrieve the disease description.
    2. **Retrieve ICD Codes:**
        a. First, attempt to use VectorSearch to find the ICD codes for the disease.
        b. If VectorSearch does not provide the ICD codes, then use TavilySearch to retrieve them.
    3. Compile the results into the specified JSON format.
    4. Once you have compiled the JSON response, you MUST output it using the "Final Answer:" prefix.

    The final answer MUST be a JSON string with the following structure:
    {{
        "disease": "<disease_name>",
        "description": "<description>",
        "icd_codes": ["<code1>", "<code2>", ...]
    }}
    If VectorSearch does not provide a description, clearly state that in the "description" field of the final JSON.
    If VectorSearch does not provide ICD codes, clearly state that in the "icd_codes" field (e.g., ["Not found in vector store, retrieved from web search"]).

    {agent_scratchpad}
    """
)

# Your agent and agent_executor initialization remains the same
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        handle_parsing_errors=True,
        max_iterations=7, # Increased max_iterations to allow for more steps (VectorSearch for desc, VectorSearch for ICD, then Tavily for ICD if needed)
        verbose=True,
        return_intermediate_steps=True,
        max_execution_time=120.0)

In [86]:
# Define the state
class AgentState(TypedDict):
    human_input: str
    messages: Sequence[HumanMessage]
    results: dict

# Define the nodes
def agent_node(state: AgentState):
    """Process the agent's response"""
    try:
        # Execute the agent
        result = agent_executor.invoke({
            "human_input": state["human_input"]
        })
        
        print("Raw result from agent:", result)
        
        # Check if the result contains the final JSON output
        if isinstance(result, dict) and "disease" in result and "description" in result and "icd_codes" in result:
            print("Final result obtained. Terminating chain.")
            return {
                "messages": state.get("messages", []),  # Keep existing messages
                "results": result  # Return the final result
            }
        
        # Continue processing if the result is incomplete
        return {
            "messages": state.get("messages", []),
            "results": result.get("output", "No output from agent")
        }
    except Exception as e:
        print(f"Error in agent_node: {e}")
        return {
            "messages": state.get("messages", []),
            "results": f"Error processing request: {str(e)}"
        }

In [87]:
# Create the graph
workflow = StateGraph(AgentState)
workflow.add_node("agent", agent_node)
workflow.set_entry_point("agent")
workflow.set_finish_point("agent")
graph = workflow.compile()

# Run the graph
def run_graph(query: str):
    """Run the agent graph with the given query"""
    inputs = {
        "human_input": query,
        "messages": [],
        "results": {}
    }
    try:
        result = graph.invoke(inputs)
        
        # Check if the result contains the final JSON output
        if isinstance(result, dict) and "disease" in result["results"] and "description" in result["results"] and "icd_codes" in result["results"]:
            print("Final result obtained. Stopping execution.")
            return result["results"]
        
        return result["results"]
    except Exception as e:
        print(f"Error running graph: {e}")
        return f"Error: {str(e)}"


In [88]:
# Generate output in JSON and CSV formats
def generate_output(query: str, result: str):
    """Convert the agent's output to JSON and CSV formats"""
    try:
        # Try to parse the result as JSON first
        try:
            # See if the result is already a valid JSON
            output = json.loads(result) if isinstance(result, str) else result
        except (json.JSONDecodeError, TypeError):
            # If not a valid JSON, try to extract JSON from text
            # Look for JSON-like structure in the text
            try:
                # Try to find JSON-like structure in text
                start_idx = result.find('{')
                end_idx = result.rfind('}')
                if start_idx >= 0 and end_idx > start_idx:
                    json_str = result[start_idx:end_idx+1]
                    output = json.loads(json_str)
                else:
                    # Fallback: create structured data from unstructured output
                    output = {
                        "disease": query,
                        "description": result.split('\n')[0] if '\n' in result else result[:100],
                        "icd_codes": []
                    }
            except Exception:
                # Last resort fallback
                output = {
                    "disease": query,
                    "description": "Failed to parse agent output",
                    "icd_codes": []
                }
        
        # Extract relevant fields with proper error handling
        disease = output.get("disease", query)
        description = output.get("description", "No description available")
        icd_codes = output.get("icd_codes", [])
        if isinstance(icd_codes, str):
            icd_codes = [icd_codes]  # Convert string to list if needed
        icd_codes_str = ", ".join(icd_codes) if icd_codes else "No ICD codes found"
        
        # Generate JSON output
        json_output = {
            "disease": disease,
            "description": description,
            "icd_codes": icd_codes
        }
        
        # Generate CSV output
        csv_output = [["Disease", "Description", "ICD Codes"]]
        csv_output.append([disease, description, icd_codes_str])
        
        return json_output, csv_output
    except Exception as e:
        print(f"Error generating output: {e}")
        # Provide a fallback output
        json_output = {"error": str(e), "query": query}
        csv_output = [["Error", "Query"], [str(e), query]]
        return json_output, csv_output

In [63]:
# Main function
def main(query: str):
    """Main function to run the agent and save results"""
    print(f"Processing query: {query}")
    result = run_graph(query)
    print(f"Result from graph: {result}")
    
    json_output, csv_output = generate_output(query, result)
    
    # Save JSON output
    with open('output.json', 'w') as f:
        json.dump(json_output, f, indent=2)
    
    # Save CSV output
    with open('output.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(csv_output)
    
    print("JSON output:", json_output)
    print("CSV output has been saved to output.csv")
    return json_output

# Example usage
if __name__ == "__main__":
    main("Gallstones")

Processing query: Gallstones


> Entering new AgentExecutor chain...
Action: VectorSearch
Action Input: Gallstones description and ICD codesStarting vector search for query: Gallstones description and ICD codes
Vector search results: [Document(id='685a0568-06b5-4b5a-be26-e0ec0ce33a87', metadata={'source': 'document3.pdf', 'page': 0, 'page_label': '1'}, page_content="Disease Information - Document3\nDisease Name: Gout\nDescription: Gout is a common and complex form of arthritis that can affect anyone. It's characterized\nby sudden, severe attacks of pain, swelling, redness and tenderness in one or more joints, most often\nin the big toe.\nICD Code: M10.9\nDisease Name: Kidney Stones\nDescription: Kidney stones (also called renal calculi, nephrolithiasis or urolithiasis) are hard deposits\nmade of minerals and salt that form inside your kidneys. Kidney stones can affect any part of your\nurinary tract.\nICD Code: N20.0\nDisease Name: Gallstones\nDescription: Gallstones are hardened depos

In [89]:
# Main function
def main(query: str):
    """Main function to run the agent and save results"""
    print(f"Processing query: {query}")
    result = run_graph(query)
    print(f"Result from graph: {result}")
    
    json_output, csv_output = generate_output(query, result)
    
    # Save JSON output
    with open('output.json', 'w') as f:
        json.dump(json_output, f, indent=2)
    
    # Save CSV output
    with open('output.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(csv_output)
    
    print("JSON output:", json_output)
    print("CSV output has been saved to output.csv")
    return json_output

# Example usage
if __name__ == "__main__":
    main("Gallstones")

Processing query: Gallstones


> Entering new AgentExecutor chain...
Thought: I need to retrieve the disease description for gallstones using VectorSearch.
Action: VectorSearch
Action Input: "gallstones description"Starting vector search for query: gallstones description
Vector search results: [Document(id='717f54e7-3be6-4619-a423-833284645f2a', metadata={'source': 'document3.pdf', 'page': 0, 'page_label': '1'}, page_content="Disease Information - Document3\nDisease Name: Gout\nDescription: Gout is a common and complex form of arthritis that can affect anyone. It's characterized\nby sudden, severe attacks of pain, swelling, redness and tenderness in one or more joints, most often\nin the big toe.\nICD Code: M10.9\nDisease Name: Kidney Stones\nDescription: Kidney stones (also called renal calculi, nephrolithiasis or urolithiasis) are hard deposits\nmade of minerals and salt that form inside your kidneys. Kidney stones can affect any part of your\nurinary tract.\nICD Code: N20.0\nDisease